<a href="https://colab.research.google.com/github/kengustafson/DeepDive-DataScience/blob/main/Wind_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pickle
import plotly.express as px
import plotly.graph_objects as go
import requests

##Functions

In [2]:
def createColumnName(lat, long):
  # Make column name - restrict numerical format to two decimal places
  columnName = str(("%.2f" % lat)) + '_' + str(("%.2f" % long))
  return columnName

In [3]:
def queryColumn(lat, long, df):
  colName = createColumnName(lat, long)
  if colName in df.columns:
    columnData = df[colName]
  else:
    columnData = []

  return columnData

In [4]:
def pred_wind_power(windspeed_df, power_curve, temp_df=None, pressure_df=None):
    """Accepts dataframe with windspeeds (floats) at a certain location, Series with power produced at defined speeds.
    Returns DataFrame of 'Month' (1-12), output power ('kW'), in whatever timeframe submitted
    (eg. hourly, monthly, etc), assuming single 2MW turbine.
    Dependent on power_curve indexes equalling defined windspeed levels, and proper curve-class passed."""

    max_curve_speed = max(power_curve.index)
    windspeed = windspeed_df['avg_windspeed']

    lower_ws = windspeed.apply(int).apply(lambda s: s if s < max_curve_speed else 0).values
    upper_ws = lower_ws+1
    power_diff = power_curve.iloc[upper_ws].reset_index(drop=True).sub(power_curve.iloc[lower_ws].reset_index(drop=True))
    frac_power = power_diff.mul(windspeed.sub(windspeed.apply(int)))
    final_power = frac_power.add(power_curve.iloc[lower_ws].reset_index(drop=True)).rename('kW')

    return pd.concat([windspeed_df, final_power], axis=1)

##Load wind data

In [5]:
#Get power curves
power_curves = pd.read_csv('/content/drive/MyDrive/Data Science Student Material/Capstone Project/Data/wind_power_curves.csv')

In [6]:
file = '/content/drive/MyDrive/Data Science Student Material/Capstone Project/Data/test/all_wind.p'
dict_allData_read = pickle.load(open(file,'rb'))
wind_df = pd.DataFrame.from_dict(dict_allData_read)
del dict_allData_read

In [7]:
file = '/content/drive/MyDrive/Data Science Student Material/Capstone Project/Data/test/upper_south_pickle.p'
dict_allData_read = pickle.load(open(file,'rb'))
upper_south_df = pd.DataFrame.from_dict(dict_allData_read)
del dict_allData_read

In [8]:
file = '/content/drive/MyDrive/Data Science Student Material/Capstone Project/Data/test/bootheel_pickle.p'
dict_allData_read = pickle.load(open(file,'rb'))
bootheel_df = pd.DataFrame.from_dict(dict_allData_read)
del dict_allData_read

In [9]:
df = pd.DataFrame(columns=['lat', 'lon', 'avg_windspeed'])

In [10]:
for el in wind_df.columns[5:]:
    tmp_df = pd.DataFrame({'lat': el.split('_')[0], 'lon': el.split('_')[1], 'avg_windspeed': wind_df[el].mean(axis=0)}, index=[0])
    df = pd.concat([df, tmp_df], axis=0, ignore_index=True)

In [11]:
for el in upper_south_df.columns[5:]:
    tmp_df = pd.DataFrame({'lat': el.split('_')[0], 'lon': el.split('_')[1], 'avg_windspeed': upper_south_df[el].mean(axis=0)}, index=[0])
    df = pd.concat([df, tmp_df], axis=0, ignore_index=True)

In [12]:
for el in bootheel_df.columns[5:]:
    tmp_df = pd.DataFrame({'lat': el.split('_')[0], 'lon': el.split('_')[1], 'avg_windspeed': bootheel_df[el].mean(axis=0)}, index=[0])
    df = pd.concat([df, tmp_df], axis=0, ignore_index=True)

In [13]:
power_df = pred_wind_power(df, power_curves['Class_2'])

In [ ]:
power_curves

In [ ]:
power_df.head()

In [19]:
"""
scl = [0,"rgb(150,0,90)"],[0.07,"rgb(0, 33, 200)"],[0.12,"rgb(0, 75, 255)"],\
[0.22,"rgb(0, 152, 255)"],[0.3,"rgb(44, 255, 150)"],[0.4,"rgb(151, 255, 0)"],\
[0.5,"rgb(255, 234, 0)"],[0.85,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]
"""
scl = [0,"rgb(255,255,255)"],[0.07,"rgb(200, 255, 255)"],[0.12,"rgb(100, 255, 255)"],\
[0.22,"rgb(151, 255, 140)"],[0.3,"rgb(151, 255, 80)"],[0.4,"rgb(151, 255, 0)"],\
[0.5,"rgb(255, 234, 0)"],[0.85,"rgb(255, 111, 0)"],[1,"rgb(255, 0, 0)"]

fig = go.Figure(data=go.Scattergeo(
    lat = power_df['lat'],
    lon = power_df['lon'],
    text = power_df['kW'].round(0).astype(str) + ' kiloWatts',
    
    marker = dict(
        color = round(power_df['kW'], 0),
        colorscale = scl,
        reversescale = False,
        opacity = 0.7,
        size = 3,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 100
        ),
        colorbar_title="Output (kW)/2MW Turbine"
    )
))

fig.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='New Mexico Potential Wind Production',
)
fig.show()
